In [1]:
import pandas as pd
import numpy as np

## Get ids, labels, some road features

In [2]:
accident_df = pd.read_csv("accidents-route.csv")

c:\Users\rahar\.conda\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (12,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
accident_df.columns

Index(['code-infraction', 'code-postal', 'date-creation-evn', 'date-fermeture',
       'date-debut-infraction', 'annee-infraction', 'mois-infraction',
       'date-infraction', 'date-fin-infraction', 'direction',
       'montant-dommage', 'numero-evenement', 'equipe', 'heure-debut',
       'heure-fin', 'secteur-ilot', 'inscription-mip', 'code-municipal',
       'nombre-personnes-dossier', 'nombre-vehicules-dossier',
       'numero-civique', 'province', 'type-rue', 'rue', 'rue-2', 'type-rue-2',
       'annee-enrg-dossier', 'mois-enrg-dossier', 'date-enrg-dossier', 'pdq',
       'vehicule-annee', 'vehicule-couleur-1', 'vehicule-genre', 'province-1',
       'vehicule-style', 'vehicule-compagnie-assurance', 'vehicule-marque',
       'vehicule-modele', 'vehicule-description'],
      dtype='object')

In [4]:
accident_df.head()

,code-infraction,code-postal,date-creation-evn,date-fermeture,date-debut-infraction,annee-infraction,mois-infraction,date-infraction,date-fin-infraction,direction,...,pdq,vehicule-annee,vehicule-couleur-1,vehicule-genre,province-1,vehicule-style,vehicule-compagnie-assurance,vehicule-marque,vehicule-modele,vehicule-description
0,Délit de fuite CSR,H7V3A2,2019-07-01,2019-07-02,2019-06-30,2019,6,30,2019-07-01,,...,PDQ3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2010.0,NaN,VUS,QUE,NaN,AVIVA,HYUN,SANT,Moyen
2,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2013.0,NaN,CAMIO,QUE,NaN,NaN,EONE,NaN,Grand
3,Collision matérielle de + de 2 000 $,NaN,2019-06-30,2019-07-05,2019-06-30,2019,6,30,NaN,,...,PDQ3,2007.0,ROU,AUTO,QUE,FGNET,INTACT,DODG,CARA,Moyen
4,Collision matérielle de + de 2 000 $,NaN,2019-06-30,2019-07-05,2019-06-30,2019,6,30,NaN,,...,PDQ3,2018.0,BLA,CAMIO,QUE,FERMEE,INTACT,GMC,SAVA,Moyen


In [27]:
for i in accident_df["numero-civique"]:
    print(i)

585
0
0
0
0
0
0
6455
176
0
0
0
0
0
1640
910
955
2955
1889
0
0
0
0
0
0
0
0
0
0
0
0
0
275
275
630
875
0
2440
0
0
0
0
7255
7255
0
0
0
2500
2500
0
0
164
164
870
4677
0
0
3025
1950
5490
0
0
2795
1038
595
0
0
47
0
0
47
0
1601
1601
3490
625
0
625
1890
2137
0
0
3000
3000
0
1238
700
0
3895
1004
3983
2850
250
0
100
629
5193
0
2228
532
1228
0
0
0
0
3735
3573
0
0
0
5
520
520
520
520
2161
2540
2161
2161
3035
0
4900
4900
4055
0
0
0
0
0
0
44
400
0
0
2475
965
0
0
370
2475
255
255
5230
5230
0
0
370
0
1536
1536
0
0
0
880
3217
3855
244
0
0
0
0
0
0
0
0
0
0
4080
0
26
2440
1006
1006
1245
3573
0
0
0
732
5436
0
0
0
0
0
0
12
1245
3035
0
1550
2440
2440
5235
265
0
0
570
570
7640
0
0
99
1641
700
1455
0
0
3200
0
0
1690
1690
0
3035
0
0
545
255
255
0
0
0
0
0
0
0
7595
364
4588
1955
1955
1501
1501
284
284
385
385
385
4125
4125
0
0
0
0
0
0
0
33
3000
0
0
2005
2005
0
0
0
0
720
720
720
0
0
3225
3035
4975
0
0
0
0
1120
0
0
4425
4425
3549
3855
3855
0
0
1545
1545
30
782
782
155
195
195
0
230
230
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [5]:
infraction_types = accident_df["code-infraction"].unique()
infraction_types

array(['Délit de fuite CSR', 'Collision matérielle de 2 000 $ et -',
       'Collision matérielle de + de 2 000 $', 'Collision avec blessé',
       'Collision mortelle', 'Collision hors route de + de 2 000 $',
       'Collision hors route avec blessé',
       'Collision hors route de 2 000 $ et -',
       'Collision mortelle hors route',
       'Incendie de véhicule en mouvement'], dtype=object)

In [6]:
collisions = ['Collision matérielle de 2 000 $ et -',
       'Collision matérielle de + de 2 000 $', 'Collision avec blessé',
       'Collision mortelle', 'Collision hors route de + de 2 000 $',
       'Collision hors route avec blessé',
       'Collision hors route de 2 000 $ et -',
       'Collision mortelle hors route']

collisions_df = accident_df[accident_df["code-infraction"].isin(collisions)]

In [7]:
print(collisions_df.shape)
collisions_df.head(3)

(24230, 39)


,code-infraction,code-postal,date-creation-evn,date-fermeture,date-debut-infraction,annee-infraction,mois-infraction,date-infraction,date-fin-infraction,direction,...,pdq,vehicule-annee,vehicule-couleur-1,vehicule-genre,province-1,vehicule-style,vehicule-compagnie-assurance,vehicule-marque,vehicule-modele,vehicule-description
1,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2010.0,NaN,VUS,QUE,NaN,AVIVA,HYUN,SANT,Moyen
2,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2013.0,NaN,CAMIO,QUE,NaN,NaN,EONE,NaN,Grand
3,Collision matérielle de + de 2 000 $,NaN,2019-06-30,2019-07-05,2019-06-30,2019,6,30,NaN,,...,PDQ3,2007.0,ROU,AUTO,QUE,FGNET,INTACT,DODG,CARA,Moyen


In [8]:
collisions_2018 = collisions_df[collisions_df["annee-infraction"] == 2018]
print(collisions_2018.shape)
collisions_2018.head(3)

(5312, 39)


,code-infraction,code-postal,date-creation-evn,date-fermeture,date-debut-infraction,annee-infraction,mois-infraction,date-infraction,date-fin-infraction,direction,...,pdq,vehicule-annee,vehicule-couleur-1,vehicule-genre,province-1,vehicule-style,vehicule-compagnie-assurance,vehicule-marque,vehicule-modele,vehicule-description
375,Collision avec blessé,NaN,2019-06-06,2019-06-11,2018-12-13,2018,12,13,NaN,,...,PDQ3,NaN,BLA,AUTO,QUE,NaN,NaN,NaN,NaN,Petit
3132,Collision matérielle de + de 2 000 $,NaN,2019-01-17,2019-01-20,2018-01-17,2018,1,17,NaN,,...,PDQ5,2018.0,NaN,VUS,QUE,CMNET,DESJARDINS,NISS,ROGU,Petit
3179,Collision avec blessé,NaN,2019-01-14,2019-01-17,2018-01-14,2018,1,14,NaN,,...,PDQ3,2013.0,NOI,AUTO,QUE,4PORT,Wawanesa,NISS,MURA,Moyen


In [9]:
collisions_2018.columns

Index(['code-infraction', 'code-postal', 'date-creation-evn', 'date-fermeture',
       'date-debut-infraction', 'annee-infraction', 'mois-infraction',
       'date-infraction', 'date-fin-infraction', 'direction',
       'montant-dommage', 'numero-evenement', 'equipe', 'heure-debut',
       'heure-fin', 'secteur-ilot', 'inscription-mip', 'code-municipal',
       'nombre-personnes-dossier', 'nombre-vehicules-dossier',
       'numero-civique', 'province', 'type-rue', 'rue', 'rue-2', 'type-rue-2',
       'annee-enrg-dossier', 'mois-enrg-dossier', 'date-enrg-dossier', 'pdq',
       'vehicule-annee', 'vehicule-couleur-1', 'vehicule-genre', 'province-1',
       'vehicule-style', 'vehicule-compagnie-assurance', 'vehicule-marque',
       'vehicule-modele', 'vehicule-description'],
      dtype='object')

In [10]:
from pyexpat import features


features = ["code-infraction","mois-infraction","type-rue","rue"]
collisions_2018 = collisions_2018[features].dropna()

In [11]:
collisions_2018.head()

,code-infraction,mois-infraction,type-rue,rue
375,Collision avec blessé,12,BOULEV,CHOMEDEY
3132,Collision matérielle de + de 2 000 $,1,BOULEV,CURE-LABELLE
3179,Collision avec blessé,1,RUE,CLARENDON
3180,Collision avec blessé,1,RUE,CLARENDON
3312,Collision matérielle de 2 000 $ et -,12,VOIE,LAVAL EST A-440


In [12]:
type_rue = collisions_2018["type-rue"].unique()
rues = collisions_2018["rue"].unique()
print(type_rue)
print(len(rues))
print(collisions_2018.shape[0]/len(rues)/12)

['BOULEV' 'RUE' 'VOIE' 'AVENUE' 'AUTORO' 'MONTEE' 'CHEMIN' 'RANG' 'ROUTE'
 'PROMEN' 'PLACE' 'TERRAS' 'ECHANG' 'VIADUC' 'PONT' 'CROISS' 'POINTE']
479
0.9241475295755045


The average for accident per road per month is almost one so month seems like a good fit for time period.

In [13]:
collisions_2018["id"] = collisions_2018["rue"] + "@" + collisions_2018["mois-infraction"].astype(str)

In [14]:
collisions_2018["id"]

375              CHOMEDEY@12
3132          CURE-LABELLE@1
3179             CLARENDON@1
3180             CLARENDON@1
3312      LAVAL EST A-440@12
                ...         
11001      LAVAL EST A-440@1
11002      DES LAURENTIDES@1
11003      DES LAURENTIDES@1
11008             SABOURIN@1
11009    LAVAL OUEST A-440@1
Name: id, Length: 5312, dtype: object

In [15]:
len(collisions_2018["id"].unique())

1162

In [16]:
uniques = collisions_2018["id"].unique()

In [17]:
# count = collisions_2018["id"].value_counts()
_count = collisions_2018.groupby(["id"])["id"].count()
print(_count.shape)
_count.head(3)

(1162,)


id
100E@10    4
100E@11    6
100E@12    7
Name: id, dtype: int64

In [18]:
uniques

array(['CHOMEDEY@12', 'CURE-LABELLE@1', 'CLARENDON@1', ...,
       'JACQUES-BUREAU@1', 'SABOURIN@1', 'DE LISBONNE@1'], dtype=object)

In [19]:
months = [int(i.split("@")[1]) for i in uniques]
road = [i.split("@")[0] for i in uniques]
road_type = [collisions_2018[collisions_2018["id"] == i]["type-rue"].iloc[0] for i in uniques]

In [20]:
count = pd.DataFrame({"id":uniques,"count":_count[uniques],"month":months,"road":road,"road_type":road_type})
count.reset_index(drop=True, inplace=True)
count.head()

,id,count,month,road,road_type
0,CHOMEDEY@12,17,12,CHOMEDEY,BOULEV
1,CURE-LABELLE@1,41,1,CURE-LABELLE,BOULEV
2,CLARENDON@1,2,1,CLARENDON,RUE
3,LAVAL EST A-440@12,11,12,LAVAL EST A-440,VOIE
4,LAVAL OUEST A-440@12,24,12,LAVAL OUEST A-440,VOIE


In [44]:
for i in rues:
    print(i)

CHOMEDEY
CURE-LABELLE
CLARENDON
LAVAL EST A-440
LAVAL OUEST A-440
ROSALIE
LEVESQUE OUEST
MARCEL-VILLENEUVE
DES LAURENTIDES
LE CORBUSIER
DE FRIBOURG
BERNARD-LEFEBVRE
100E
BRAHMS
DE NORMANDIE
RENE-LAENNEC
LOUIS-DUROCHER
SAINT-MARTIN OUEST
77E
VINCENT-MASSEY
AUBERT
BELLEROSE OUEST
CHOMEDEY A-13
DAGENAIS OUEST
LAVAL
ALBERT-MURPHY
DU SOUVENIR
PAPINEAU A-19
JOLIVET
GUSTAVE
DES PERRON
DE PROVENCE
NOTRE-DAME-DE-FATIMA
SAMSON
DE LA CONCORDE EST
DES LAURENTIDES A-15
DAGENAIS EST
SAINT-SAENS EST
DE GALAIS
ROBERT-ELIE
CARTIER OUEST
DANIEL-JOHNSON
PIERRE-CHASSEUR
DE LA STATION
DES PRAIRIES
SEIGNEUR-LUSSIER
DE CHENONCEAU
SAINTE-ROSE
INDUSTRIEL
LE CARREFOUR
MASSON
NOTRE-DAME
JULES-LAFRENIERE
COUSINEAU
VANIER
CUNARD
ERNEST-CORMIER
ST MARTIN
TREPANIER
EMILE
DES MILLE-ILES
CHARTRAND
15E
JEAN-BERAUD
LOUIS-PAYETTE
PRINCIPALE
ROBIN
DE LA LAVATERE
SAINT-FRANCOIS
AMPERE
DELISLE
SAINT-ELZEAR OUEST
DE BOIS-DE-BOULOGNE
DE L EQUERRE
DE GENEVE
CHARTIER
DU VAL-MARTIN
PAPINEAU
ISLEMERE
PASCAL
CHARLOTTE
D AUTEUIL
BI

## Getting speed limits in the df

In [59]:
speed_raw = pd.read_csv("limite-vitesse-2014.csv")
speed_raw.head(5)

,WKT,ID_TRC,T_VOIE,L_VOIE,N_VOIE,D_VOIE,LI_GCH,LI_DRT,ODONYME,ODONYME_CO,LIMITE_VIT
0,LINESTRING (-73.6798629184971 45.5559610468134...,508381,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
1,LINESTRING (-73.6806608913008 45.5555611608213...,508380,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
2,LINESTRING (-73.6814206363064 45.5552261180531...,508379,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
3,LINESTRING (-73.6822380588474 45.5549062754073...,508378,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
4,LINESTRING (-73.6830030082074 45.5546018081614...,508377,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50


In [67]:
types_voies = speed_raw["T_VOIE"].unique()
types_voies = [i.lower() for i in types_voies]
types_voies

['boulevard',
 'rang',
 'montée',
 'rue',
 'place',
 'avenue',
 'desserte sud',
 'desserte est',
 'desserte ouest',
 'desserte nord',
 'autoroute',
 'échangeur',
 'pont',
 'chemin',
 'terrasse',
 'jardin',
 'croissant',
 'viaduc',
 'promenade',
 'court',
 'esplanade']

In [73]:
count_types = ['BOULEV', 'RUE' 'VOIE', 'AVENUE', 'AUTORO', 'MONTEE', 'CHEMIN', 'RANG', 'ROUTE',
 'PROMEN', 'PLACE', 'TERRAS', 'ECHANG', 'VIADUC', 'PONT', 'CROISS', 'POINTE']

type_dict = {i:i.lower() for i in count_types if i.lower() in types_voies}
type_dict
for i in count_types:
    if i.lower() not in  types_voies:
        print(i)

BOULEV
RUEVOIE
AUTORO
MONTEE
ROUTE
PROMEN
TERRAS
ECHANG
CROISS
POINTE


In [ ]:
type_dict = {"BOULEV":"boulevard","RUEVOIE"}

In [32]:
odonymes = speed_raw["ODONYME"].unique()
print(len(odonymes))
print(odonymes[0:5])

3023
['Boulevard des Prairies' 'Boulevard Cléroux' "Rang de l'Équerre"
 'Montée Major' "Rue d'Aurillac"]


In [33]:
check = []
welp = []
for i in rues:
    for j in odonymes:
        if i.lower() in j.lower():
            check.append(True)
            welp.append(True)
            break
    check.append(False)

In [34]:
check

[True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,

In [31]:
len(welp)

363

In [47]:
n_voies = speed_raw["N_VOIE"].unique()
print(len(n_voies))
check = []
welp = []
for i in n_voies:
    for j in rues:
        if i.lower() in j.lower():
            check.append(True)
            welp.append(True)
    check.append(False)

2779


In [48]:
for i in n_voies:
    print(i)

Prairies
Cléroux
Équerre
Major
Aurillac
Jean-Deslauriers
Monterey
Saint-François
Desrochers
Léon
Donck
Nevers
Labelle
Dinan
Pont-Viau
Saint-Elzéar
Émile
Saulnier
Lacasse
Juge-Desnoyers
Saint-Hubert
Guilbault
Conrad
Perron
Francis-Hughes
Laplante
Micro
Pontmain
Laval Ouest (A-440)
Chomedey (A-13)
Laurentides (A-15)
Chomedey (A-13)-Saint-Martin
Chomedey (A-13)-Samson
Laurentides (A-15)-Laval (A-440)
Papineau (A-19)-Saint-Martin
A-25
Papineau (A-19)
Laurentides
Talbot
Notre-Dame
Bois
Samson
Leblanc
Le Corbusier
Papineau
Jacques-Bureau
Viau
Élodie-Boucher
Orly
Principale
15e
Tour
Bord-de-l'Eau
Relais
Fontenelle
Val-Brillant
Grande-Anse
Hector-Nadon
Chemin-de-Fer
Maurice-Lalonde
Marois
Jeanne
Jocelyne
Marie-Lefranc
Anse-Pleureuse
Islemère
Saint-Antoine
Rocailles
Trilles
Comtois
Plaines
Cèdres
Peupliers
13e
38e
7e
Pierre-Emmanuel
Albert-Camus
21e
Fernando-Pessoa
80e
47e
49e
Brady
Matane
Anse-à-Beaufils
Jeanne-d'Arc
Simard
Lafontaine
Dupont
Beaudry
Poètes
9e
3e
Érables
Sorbiers
Île-Roussin
1r

In [49]:
print(len(welp)/len(rues))

1.104384133611691


The second approach seems to offer a good correspondance (91.4405%)
Ok now it'S 110% so there are double matches

In [56]:
welp = []
for index,row in speed_raw.iterrows():
    # if row[4] not in d.keys():
    #     d[row[4]] = row[2]
    lim_name = row[4]
    lim_type = row[2]
    for j,k in count.iterrows():
        c_name = k[3]
        c_type = k[4]
